In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import json

# Cấu hình Selenium để sử dụng trình duyệt không giao diện
options = Options()
options.headless = True  # Chạy không giao diện

# Khởi tạo trình duyệt
driver = webdriver.Chrome(options=options)

# Mở trang web
url = "https://food.grab.com/vn/vi/"
driver.get(url)

# Đợi trang tải (có thể cần điều chỉnh thời gian chờ)
time.sleep(5)

# Lấy nội dung HTML sau khi trang đã tải
html_content = driver.page_source

# Phân tích HTML với BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Danh sách lưu dữ liệu
categories = []

# Tìm tất cả các mục div chứa class "ant-col-12"
category_items = soup.find_all('div', class_="ant-col-12")
print(f"Tìm thấy {len(category_items)} danh mục")

for item in category_items:
    link_tag = item.find('a', role="button")
    href = link_tag['href'] if link_tag else None
    title_tag = item.find('div', class_="title___mFwYG")
    title = title_tag.get_text(strip=True) if title_tag else None
    img_tag = item.find('img', class_="realImage___2TyNE")
    img_url = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None

    if title and href and img_url:
        categories.append({
            "name": title,
            "link": href,
            "image_url": img_url
        })

# Lưu dữ liệu vào file JSON
output_file = 'grab_categories.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được lưu vào file '{output_file}'")

# Đóng trình duyệt sau khi hoàn thành
driver.quit()


Tìm thấy 17 danh mục
Dữ liệu đã được lưu vào file 'grab_categories.json'


In [12]:
from bs4 import BeautifulSoup
import requests
import json

# URL của trang web bạn muốn crawl
url = "https://shopeefood.vn/"

# Gửi yêu cầu GET đến trang web
try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Kiểm tra nếu có lỗi HTTP
except requests.exceptions.RequestException as e:
    print(f"Lỗi khi kết nối tới {url}: {e}")
    exit()

# Phân tích HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Danh sách lưu dữ liệu
categories = []

# Tìm tất cả các thẻ <a> trong danh mục
category_links = soup.select('.category-list-filter a')  # Sử dụng `select` với CSS Selector
for link in category_links:
    # Lấy tên danh mục
    name = link.get_text(strip=True)
    # Lấy URL
    href = link.get('href', '#')  # Mặc định nếu không có href

    # Thêm dữ liệu vào danh sách
    categories.append({
        "name": name,
        "link": f"https://shopeefood.vn{href}" if href.startswith('/') else href
    })

# Lưu dữ liệu vào file JSON
output_file = 'shopee.json'
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(categories, f, ensure_ascii=False, indent=4)
    print(f"Dữ liệu đã được lưu vào file '{output_file}'")
except IOError as e:
    print(f"Lỗi khi ghi file '{output_file}': {e}")


Dữ liệu đã được lưu vào file 'shopee.json'


In [ ]:
from bs4 import BeautifulSoup
import requests
import json

# URL của trang web bạn muốn crawl
url = "https://www.foody.vn/ha-noi"

# Gửi yêu cầu GET đến trang web
try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()  # Kiểm tra nếu có lỗi HTTP
    html_content = response.text  # Lấy nội dung HTML của trang
except requests.exceptions.RequestException as e:
    print(f"Lỗi khi kết nối tới {url}: {e}")
    exit()

# Phân tích HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Danh sách lưu dữ liệu
categories = []

# Tìm tất cả các mục <li> chứa danh mục (Dựa vào cấu trúc thực tế)
list_items = soup.find_all('li', recursive=True)
for li in list_items:
    # Tìm thẻ <span> chứa tên danh mục
    span = li.find('span')
    if span:
        name = span.get_text(strip=True)
        # Thêm dữ liệu vào danh sách
        categories.append({"name": name})

# Lưu dữ liệu vào file JSON
output_file = 'foody.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được lưu vào file '{output_file}'")


Dữ liệu đã được lưu vào file 'category_list.json'


In [ ]:
import psycopg2
import json

# Thiết lập kết nối đến PostgreSQL
conn = psycopg2.connect(
    dbname="food_delivery", 
    user="your_username", 
    password="your_password", 
    host="localhost", 
    port="5432"
)
cursor = conn.cursor()

# Hàm để lưu dữ liệu vào bảng PostgreSQL
def save_to_db(table, columns, values):
    insert_query = f"""
    INSERT INTO {table} ({', '.join(columns)}) 
    VALUES ({', '.join(['%s'] * len(values))});
    """
    cursor.execute(insert_query, values)

# Đọc dữ liệu từ file JSON
with open('restaurants.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Duyệt qua từng nhà hàng và lưu dữ liệu vào PostgreSQL
for restaurant in data:
    # Lưu nhà hàng vào bảng restaurants
    restaurant_columns = ['name', 'address', 'status']
    restaurant_values = [restaurant['restaurant_name'], restaurant['address'], 'open']
    save_to_db('restaurants', restaurant_columns, restaurant_values)
    
    # Lấy restaurant_id vừa được tạo
    cursor.execute("SELECT LASTVAL();")
    restaurant_id = cursor.fetchone()[0]

    # Lưu món ăn vào bảng menu_items
    for item in restaurant['menu_items']:
        menu_item_columns = ['restaurant_id', 'name', 'price', 'description', 'status']
        menu_item_values = [restaurant_id, item['name'], float(item['price']), item['description'], 'available']
        save_to_db('menu_items', menu_item_columns, menu_item_values)

# Commit thay đổi và đóng kết nối
conn.commit()
cursor.close()
conn.close()
print("Dữ liệu đã được tải lên PostgreSQL thành công!")